In [1]:
#NN on new york data set without normalizing the output and without merging latitudes and longitudes to predict the **duration** 
#Project Title: Improving the Bus Transit System by Predicting the Bus Arrival time, Using Regression Models and Neural Networks
# Group Members: Sanaz Hossseini, Shruthi Pasumarthi, Narges Rashvand

In [1]:
import os
import time
from datetime import datetime
import torch
import numpy as np
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch.nn as nn
import torch.nn.functional as F
torch.set_printoptions(edgeitems=2, linewidth=75)
import math

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
import pandas as pd

In [2]:
file = "MTA-1706-Edited-12112022.csv"
target_file = os.getcwd() + f"/{file}"

In [3]:
df = pd.read_csv(target_file)
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
0,0:03:00,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,0:03:00,0:06:14
1,0:03:00,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,0:03:00,23:58:02
2,0:03:00,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,0:03:00,0:00:53
3,0:03:00,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,0:04:00,0:03:00
4,0:03:00,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,0:03:00,23:59:38
...,...,...,...,...,...,...,...,...,...,...
9995,5:03:00,40.622311,-74.028687,40.640297,-74.131332,40.596955,-74.069769,90.0,5:03:00,5:02:12
9996,5:03:00,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,5:04:00,4:22:13
9997,5:03:00,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,5:04:00,4:20:25
9998,5:03:00,40.862228,-73.890442,40.877934,-73.866585,40.871502,-73.876412,185.0,5:03:00,5:01:25


In [4]:
df.dropna(axis = 0, how = 'any', thresh = None, subset = None, inplace = True) #The code, if we want to remove all the rows with empty cells.  
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
0,0:03:00,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,0:03:00,0:06:14
1,0:03:00,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,0:03:00,23:58:02
2,0:03:00,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,0:03:00,0:00:53
3,0:03:00,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,0:04:00,0:03:00
4,0:03:00,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,0:03:00,23:59:38
...,...,...,...,...,...,...,...,...,...,...
9995,5:03:00,40.622311,-74.028687,40.640297,-74.131332,40.596955,-74.069769,90.0,5:03:00,5:02:12
9996,5:03:00,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,5:04:00,4:22:13
9997,5:03:00,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,5:04:00,4:20:25
9998,5:03:00,40.862228,-73.890442,40.877934,-73.866585,40.871502,-73.876412,185.0,5:03:00,5:01:25


In [5]:
print(df.dtypes)

RecordedAtTime                object
OriginLat                    float64
OriginLong                   float64
DestinationLat               float64
DestinationLong              float64
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
DistanceFromStop             float64
ExpectedArrivalTime           object
ScheduledArrivalTime          object
dtype: object


In [6]:
print(df['RecordedAtTime'])
print(df['ExpectedArrivalTime'])
print(df['ScheduledArrivalTime'])

0       0:03:00
1       0:03:00
2       0:03:00
3       0:03:00
4       0:03:00
         ...   
9995    5:03:00
9996    5:03:00
9997    5:03:00
9998    5:03:00
9999    5:03:00
Name: RecordedAtTime, Length: 8933, dtype: object
0       0:03:00
1       0:03:00
2       0:03:00
3       0:04:00
4       0:03:00
         ...   
9995    5:03:00
9996    5:04:00
9997    5:04:00
9998    5:03:00
9999    5:03:00
Name: ExpectedArrivalTime, Length: 8933, dtype: object
0        0:06:14
1       23:58:02
2        0:00:53
3        0:03:00
4       23:59:38
          ...   
9995     5:02:12
9996     4:22:13
9997     4:20:25
9998     5:01:25
9999     5:04:00
Name: ScheduledArrivalTime, Length: 8933, dtype: object


In [7]:
df['RecordedAtTime'] = pd.to_datetime(df['RecordedAtTime'], errors = 'coerce')
tindex = pd.DatetimeIndex(df.RecordedAtTime) 
df['RecordedAtTime'] = tindex.strftime('%H:%M:%S')

In [8]:
df['RecordedAtTime'] = pd.to_timedelta(df['RecordedAtTime']).dt.total_seconds()
df['RecordedAtTime'] 

0         180.0
1         180.0
2         180.0
3         180.0
4         180.0
         ...   
9995    18180.0
9996    18180.0
9997    18180.0
9998    18180.0
9999    18180.0
Name: RecordedAtTime, Length: 8933, dtype: float64

In [9]:
df['ExpectedArrivalTime'] = pd.to_datetime(df['ExpectedArrivalTime'], errors = 'coerce')
tindex2 = pd.DatetimeIndex(df.ExpectedArrivalTime) 
df['ExpectedArrivalTime'] = tindex2.strftime('%H:%M:%S')

In [10]:
df['ExpectedArrivalTime'] = pd.to_timedelta(df['ExpectedArrivalTime']).dt.total_seconds()
df['ExpectedArrivalTime'] 

0         180.0
1         180.0
2         180.0
3         240.0
4         180.0
         ...   
9995    18180.0
9996    18240.0
9997    18240.0
9998    18180.0
9999    18180.0
Name: ExpectedArrivalTime, Length: 8933, dtype: float64

In [11]:
df['ScheduledArrivalTime'] = pd.to_datetime(df['ScheduledArrivalTime'], errors = 'coerce')
tindex3 = pd.DatetimeIndex(df.ScheduledArrivalTime) 
df['ScheduledArrivalTime'] = tindex3.strftime('%H:%M:%S')

In [12]:
df['ScheduledArrivalTime'] = pd.to_timedelta(df['ScheduledArrivalTime']).dt.total_seconds()
df['ScheduledArrivalTime'] 

0         374.0
1       86282.0
2          53.0
3         180.0
4       86378.0
         ...   
9995    18132.0
9996    15733.0
9997    15625.0
9998    18085.0
9999    18240.0
Name: ScheduledArrivalTime, Length: 8933, dtype: float64

In [13]:
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime
0,180.0,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,180.0,374.0
1,180.0,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,180.0,86282.0
2,180.0,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,180.0,53.0
3,180.0,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,240.0,180.0
4,180.0,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,180.0,86378.0
...,...,...,...,...,...,...,...,...,...,...
9995,18180.0,40.622311,-74.028687,40.640297,-74.131332,40.596955,-74.069769,90.0,18180.0,18132.0
9996,18180.0,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,18240.0,15733.0
9997,18180.0,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,18240.0,15625.0
9998,18180.0,40.862228,-73.890442,40.877934,-73.866585,40.871502,-73.876412,185.0,18180.0,18085.0


In [14]:
print(df.dtypes)

RecordedAtTime               float64
OriginLat                    float64
OriginLong                   float64
DestinationLat               float64
DestinationLong              float64
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
DistanceFromStop             float64
ExpectedArrivalTime          float64
ScheduledArrivalTime         float64
dtype: object


In [15]:
df['duration']=abs(df['ExpectedArrivalTime'] - df['RecordedAtTime'])
df

,RecordedAtTime,OriginLat,OriginLong,DestinationLat,DestinationLong,VehicleLocation.Latitude,VehicleLocation.Longitude,DistanceFromStop,ExpectedArrivalTime,ScheduledArrivalTime,duration
0,180.0,40.616104,-74.031143,40.656048,-73.907379,40.635170,-73.960803,76.0,180.0,374.0,0.0
1,180.0,40.643169,-74.073494,40.575935,-74.167686,40.590802,-74.158340,62.0,180.0,86282.0,0.0
2,180.0,40.875008,-73.880142,40.912376,-73.902534,40.886010,-73.912647,5.0,180.0,53.0,0.0
3,180.0,40.701748,-73.802399,40.666012,-73.735939,40.668002,-73.729348,267.0,240.0,180.0,60.0
4,180.0,40.881187,-73.909340,40.809654,-73.928360,40.868134,-73.893032,11.0,180.0,86378.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
9995,18180.0,40.622311,-74.028687,40.640297,-74.131332,40.596955,-74.069769,90.0,18180.0,18132.0,0.0
9996,18180.0,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,18240.0,15733.0,60.0
9997,18180.0,40.761806,-73.829559,40.692406,-73.745216,40.692326,-73.744289,318.0,18240.0,15625.0,60.0
9998,18180.0,40.862228,-73.890442,40.877934,-73.866585,40.871502,-73.876412,185.0,18180.0,18085.0,0.0


In [16]:
print(df.dtypes)

RecordedAtTime               float64
OriginLat                    float64
OriginLong                   float64
DestinationLat               float64
DestinationLong              float64
VehicleLocation.Latitude     float64
VehicleLocation.Longitude    float64
DistanceFromStop             float64
ExpectedArrivalTime          float64
ScheduledArrivalTime         float64
duration                     float64
dtype: object


In [17]:
X = df.iloc[:, 0:8].values
Y = df.iloc[:, 10].values
Y

array([ 0.,  0.,  0., ..., 60.,  0.,  0.])

In [18]:
X[0]

array([180.      ,  40.616104, -74.031143,  40.656048, -73.907379,
        40.63517 , -73.960803,  76.      ])

In [19]:
Y.shape

(8933,)

In [20]:
X.shape

(8933, 8)

In [21]:
scaler = StandardScaler()
X= scaler.fit_transform(X)
X

array([[-1.22280821, -1.16239472, -0.97730579, ..., -0.93302388,
        -0.30146583, -0.2364487 ],
       [-1.22280821, -0.85492136, -1.39596011, ..., -1.44540336,
        -2.33698975, -0.27106871],
       [-1.22280821,  1.77889781,  0.51539147, ...,  1.96377721,
         0.19475862, -0.41202159],
       ...,
       [ 1.88498252,  0.49286052,  1.01542197, ..., -0.27296343,
         1.92960696,  0.36198284],
       [ 1.88498252,  1.63370995,  0.4135724 , ...,  1.796233  ,
         0.5681429 ,  0.03309278],
       [ 1.88498252, -0.15355492,  1.37381522, ...,  0.62418571,
         1.08177168, -0.42438588]])

In [22]:
X = torch.tensor(X)
Y = torch.tensor(Y)
X.shape

torch.Size([8933, 8])

In [23]:
Y.shape

torch.Size([8933])

In [24]:
n_samples = X.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_X = X[train_indices]
train_Y = Y[train_indices]
train_Y = train_Y.unsqueeze(1)

val_X = X[val_indices]
val_Y = Y[val_indices]
val_Y = val_Y.unsqueeze(1)

In [25]:
print(train_X.shape)
print(train_Y.shape)
print(val_X.shape)
print(val_Y.shape)

torch.Size([7147, 8])
torch.Size([7147, 1])
torch.Size([1786, 8])
torch.Size([1786, 1])


In [26]:
train_X = train_X.float()
train_Y = train_Y.float()
val_X = val_X.float()
val_Y = val_Y.float()
print(train_X.dtype)
print(train_Y.dtype)
print(val_X.dtype)
print(val_Y.dtype)

torch.float32
torch.float32
torch.float32
torch.float32


In [27]:
model = nn.Sequential(
            nn.Linear(8, 512), 
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 10),
            nn.ReLU(),
            nn.Linear(10, 4),
            nn.ReLU(),
            nn.Linear(4, 1))

model

Sequential(
  (0): Linear(in_features=8, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ReLU()
  (6): Linear(in_features=128, out_features=64, bias=True)
  (7): ReLU()
  (8): Linear(in_features=64, out_features=10, bias=True)
  (9): ReLU()
  (10): Linear(in_features=10, out_features=4, bias=True)
  (11): ReLU()
  (12): Linear(in_features=4, out_features=1, bias=True)
)

In [28]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_X, val_X, train_Y, val_Y):
    
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_X) 
        train_loss = loss_fn(train_t_p, train_Y)
                             
        val_t_p = model(val_X) 
        val_loss = loss_fn(val_t_p, val_Y)
        
        optimizer.zero_grad()
        train_loss.backward() 
        optimizer.step()

        if epoch ==1 or epoch % 500==0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")

In [29]:
optimizer= optim.Adam(model.parameters(), lr=1e-4)

training_loop(
    n_epochs=15000,
    optimizer= optimizer,
    model=model,
    loss_fn=nn.MSELoss(),
    train_X=train_X,
    val_X= val_X,
    train_Y= train_Y,
    val_Y= val_Y)

Epoch 1, Training loss 4268.8999, Validation loss 4059.5806
Epoch 500, Training loss 1422.9360, Validation loss 1329.6038
Epoch 1000, Training loss 1145.1362, Validation loss 1253.7794
Epoch 1500, Training loss 1050.2256, Validation loss 1197.0659
Epoch 2000, Training loss 882.8903, Validation loss 1150.5685
Epoch 2500, Training loss 731.3357, Validation loss 1133.8334
Epoch 3000, Training loss 627.9051, Validation loss 1147.8530
Epoch 3500, Training loss 561.5905, Validation loss 1174.1500
Epoch 4000, Training loss 512.5338, Validation loss 1186.1508
Epoch 4500, Training loss 471.7755, Validation loss 1214.0040
Epoch 5000, Training loss 440.7549, Validation loss 1229.7946
Epoch 5500, Training loss 416.6409, Validation loss 1239.3130
Epoch 6000, Training loss 394.2462, Validation loss 1286.2872
Epoch 6500, Training loss 371.7189, Validation loss 1277.1963
Epoch 7000, Training loss 355.9491, Validation loss 1285.7655
Epoch 7500, Training loss 338.2343, Validation loss 1314.1396
Epoch 80